In [13]:
import pandas as pd
import os
from astropy.table import Table
import numpy as np

obj_56 = {'name': 'HOPS-56', 'frequency': 33.0e9, 'flux':5.69e-5}
obj_65 = {'name': 'HOPS-65', 'frequency': 15.0e9, 'flux':1.064e-5}
obj_124 = {'name': 'HOPS-124', 'frequency': 44.0e9, 'flux': 0.0005424}
obj_140 = {'name': 'HOPS-140', 'frequency': 33.0e9, 'flux':0.0001537}
obj_157_a = {'name': 'HOPS-157_a', 'frequency': 33.0e9, 'flux':2.67e-5}
obj_157_b = {'name': 'HOPS-157_b', 'frequency': 33.0e9, 'flux':2.34e-5}
obj_163_3RMS = {'name': 'HOPS-163_3RMS', 'frequency': 33.0e9, 'flux':4.87e-6*3}
obj_270_3RMS = {'name': 'HH270MMS2_3RMS', 'frequency': 44.0e9, 'flux':8.75e-6*3}

obj_56_3RMS = {'name': 'HOPS-56_3RMS', 'frequency': 33.0e9, 'flux':7.81e-6*3}
obj_65_3RMS = {'name': 'HOPS-65_3RMS', 'frequency': 15.0e9, 'flux':2.44e-6*3}
obj_124_3RMS = {'name': 'HOPS-124_3RMS', 'frequency': 44.0e9, 'flux': 6.54e-5*3}
obj_140_3RMS = {'name': 'HOPS-140_3RMS', 'frequency': 33.0e9, 'flux':7.54e-6*3}
obj_157_3RMS = {'name': 'HOPS-157_3RMS', 'frequency': 33.0e9, 'flux':9.49e-6*3}

objs = [obj_56, obj_56_3RMS, obj_65, obj_65_3RMS, obj_124, obj_124_3RMS, obj_56_3RMS, obj_140, obj_140_3RMS, obj_157_a, obj_157_b, obj_157_3RMS, obj_163_3RMS, obj_270_3RMS]

tracs = ["BHAC15","Siess2000",  "Dotter2008", "Tognelli2011","Feiden2016","Feiden2016mag","Chen2014","Bressan2012"]


In [ ]:
def read_mass_data(direc, file_name):
    df = pd.read_csv(os.path.join(direc, file_name))
    return df

def calculate_statistic_properties(data, center, percentile_u, percentile_l):
    data_c = np.percentile(data,center)
    data_u = np.percentile(data,percentile_u) - data_c
    data_l = data_c - np.percentile(data,percentile_l)
    return [data_c, data_u, data_l]

In [32]:
def mass_table_with_upper_lower_limit(condition='prop056_05myr', mass_table_direc='data_result/mass_table', with_flux=False):    
    t = Table()
    
    t['object'] = [objs[i]['name'] for i in range(len(objs))]
    if with_flux:
        t['flux'] = [f'{objs[i]["flux"]:.3e}' for i in range(len(objs))]

    value_per_condition = []
    for obj in objs:
        value_per_obj = []
        file_name = f'{obj["name"]}_mass_table_{condition}.csv'
        df = read_mass_data(mass_table_direc, file_name)
        for track in tracks: 
            value_per_obj.append(calculate_statistic_properties(df[track], 50, 95, 5))

        value_per_condition.append(value_per_obj)

    for track_j in range(len(value_per_condition[0])):  #value_per_condition[0]: first object
        new_column = []
        column_name = tracks[track_j]
        for obj_i in range(len(value_per_condition)):
            value = value_per_condition[obj_i][track_j]
            table_value = "$"+f'{value[0]:.2f}'+"^{+"+f'{value[1]:.2f}'+"}_{-"+\
                        f'{value[2]:.2f}'+"}$"
            new_column.append(table_value)
        t[column_name]=new_column

    return t


In [45]:
def write_latex_table(table, condition, data_origin):
    direc = 'data_result/mass_latex_table'
    os.makedirs(direc, exist_ok=True)
    
    if data_origin == 'data':
        if condition == 'prop056_05myr':
            table.write("data_result/mass_latex_table/mass_table_prop056_05myr.tex", format="ascii.aastex", latexdict={\
                        'tabletype':'deluxetable*',
                        'preamble':[r'\tablecaption{Interporlated mass(0.5 Myr and $L_{bol} \propto v^{0.56}$)}',\
                        r'\tablenum{1}',r'\tabletypesize{\normalsize}', \
                        r'\label{table: 0.5myr 0.56}'],\
                        'col_align':'l|cccccccc'}, \
                        overwrite=True)


        elif condition == 'prop056_1myr':
             table.write("data_result/mass_latex_table/mass_table_prop056_1myr.tex", format="ascii.aastex", latexdict={\
                         'tabletype':'deluxetable*',
                         'preamble':[r'\tablecaption{Interporlated mass(1 Myr and $L_{bol} \propto v^{0.56}$)}',\
                         r'\tablenum{2}',r'\tabletypesize{\normalsize}', \
                         r'\label{table: 1myr 0.56}'],\
                         'col_align':'l|cccccccc'}, \
                         overwrite=True)

        elif condition == 'propn1_05myr':
             table.write("data_result/mass_latex_table/mass_table_propn1_05myr.tex", format="ascii.aastex", latexdict={\
                         'tabletype':'deluxetable*',
                         'preamble':[r'\tablecaption{Interporlated mass(0.5 Myr and $L_{bol} \propto v^{-1}$)}',\
                         r'\tablenum{3}',r'\tabletypesize{\normalsize}', \
                         r'\label{table: 0.5myr -1}'],\
                         'col_align':'l|cccccccc'}, \
                         overwrite=True)    

        elif condition == 'propn1_1myr':
             table.write("data_result/mass_latex_table/mass_table_propn1_1myr.tex", format="ascii.aastex", latexdict={\
                         'tabletype':'deluxetable*',
                         'preamble':[r'\tablecaption{Interporlated mass(1 Myr and $L_{bol} \propto v^{-1}$)}',\
                         r'\tablenum{4}',r'\tabletypesize{\normalsize}', \
                         r'\label{table: 1myr -1}'],\
                         'col_align':'l|cccccccc'}, \
                         overwrite=True)
        else: 
            raise Exception("Illegal condition")
    
    
    elif data_origin == 'ngVLA':
        if condition == 'prop056_05myr':
            table.write("data_result/mass_latex_table/mass_table_prop056_05myr_ngVLA.tex", format="ascii.aastex", latexdict={\
                        'tabletype':'deluxetable*',
                        'preamble':[r'\tablecaption{Interporlated mass for ngVLA limit(0.5 Myr and $L_{bol} \propto v^{0.56}$)}',\
                        r'\tablenum{1}',r'\tabletypesize{\normalsize}', \
                        r'\label{table: 0.5myr 0.56}'],\
                        'col_align':'l|cccccccc'}, \
                        overwrite=True)


        elif condition == 'prop056_1myr':
             table.write("data_result/mass_latex_table/mass_table_prop056_1myr_ngVLA.tex", format="ascii.aastex", latexdict={\
                         'tabletype':'deluxetable*',
                         'preamble':[r'\tablecaption{Interporlated mass for ngVLA limit(1 Myr and $L_{bol} \propto v^{0.56}$)}',\
                         r'\tablenum{2}',r'\tabletypesize{\normalsize}', \
                         r'\label{table: 1myr 0.56}'],\
                         'col_align':'l|cccccccc'}, \
                         overwrite=True)

        elif condition == 'propn1_05myr':
             table.write("data_result/mass_latex_table/mass_table_propn1_05myr_ngVLA.tex", format="ascii.aastex", latexdict={\
                         'tabletype':'deluxetable*',
                         'preamble':[r'\tablecaption{Interporlated mass for ngVLA limit(0.5 Myr and $L_{bol} \propto v^{-1}$)}',\
                         r'\tablenum{3}',r'\tabletypesize{\normalsize}', \
                         r'\label{table: 0.5myr -1}'],\
                         'col_align':'l|cccccccc'}, \
                         overwrite=True)    

        elif condition == 'propn1_1myr':
             table.write("data_result/mass_latex_table/mass_table_propn1_1myr_ngVLA.tex", format="ascii.aastex", latexdict={\
                         'tabletype':'deluxetable*',
                         'preamble':[r'\tablecaption{Interporlated mass for ngVLA limit(1 Myr and $L_{bol} \propto v^{-1}$)}',\
                         r'\tablenum{4}',r'\tabletypesize{\normalsize}', \
                         r'\label{table: 1myr -1}'],\
                         'col_align':'l|cccccccc'}, \
                         overwrite=True)  
        else: 
            raise Exception("Illegal condition")

    
    
    else:
        raise Exception("Illegal data origin")

In [47]:
mass_table_with_upper_lower_limit(condition='prop056_1myr', mass_table_direc='data_result/mass_table', with_flux=True)

object,flux,BHAC15,Siess2000,Dotter2008,Tognelli2011,Feiden2016,Feiden2016mag,Chen2014,Bressan2012
str14,str9,str22,str22,str22,str22,str22,str22,str22,str22
HOPS-56,5.690e-05,$1.03^{+0.24}_{-0.17}$,$0.91^{+0.24}_{-0.23}$,$1.03^{+0.23}_{-0.17}$,$0.84^{+0.29}_{-0.24}$,$0.95^{+0.23}_{-0.16}$,$1.23^{+0.20}_{-0.18}$,$0.97^{+0.23}_{-0.17}$,$0.97^{+0.23}_{-0.17}$
HOPS-56_3RMS,2.343e-05,$0.55^{+0.13}_{-0.10}$,$0.39^{+0.17}_{-0.10}$,$0.59^{+0.12}_{-0.10}$,$0.37^{+0.07}_{-0.05}$,$0.50^{+0.13}_{-0.14}$,$0.66^{+0.17}_{-0.17}$,$0.57^{+0.11}_{-0.09}$,$0.52^{+0.12}_{-0.09}$
HOPS-65,1.064e-05,$0.41^{+0.10}_{-0.08}$,$0.28^{+0.02}_{-0.04}$,$0.45^{+0.10}_{-0.08}$,$0.30^{+0.05}_{-0.04}$,$0.27^{+0.18}_{-0.04}$,$0.37^{+0.22}_{-0.11}$,$0.43^{+0.10}_{-0.09}$,$0.39^{+0.10}_{-0.09}$
HOPS-65_3RMS,7.320e-06,$0.30^{+0.08}_{-0.08}$,$0.22^{+0.05}_{-0.03}$,$0.35^{+0.08}_{-0.06}$,$0.24^{+0.04}_{-0.04}$,$0.22^{+0.04}_{-0.04}$,$0.25^{+0.12}_{-0.04}$,$0.30^{+0.10}_{-0.09}$,$0.27^{+0.09}_{-0.07}$
HOPS-124,5.424e-04,$nan^{+nan}_{-nan}$,$3.23^{+0.26}_{-0.32}$,$3.15^{+0.20}_{-0.31}$,$3.28^{+0.22}_{-0.35}$,$3.16^{+0.21}_{-0.30}$,$nan^{+nan}_{-nan}$,$3.14^{+0.20}_{-0.31}$,$3.14^{+0.21}_{-0.31}$
HOPS-124_3RMS,1.962e-04,$nan^{+nan}_{-nan}$,$2.05^{+0.51}_{-0.41}$,$2.10^{+0.45}_{-0.37}$,$2.08^{+0.51}_{-0.42}$,$2.06^{+0.48}_{-0.42}$,$1.70^{+0.00}_{-0.00}$,$2.05^{+0.47}_{-0.38}$,$2.05^{+0.48}_{-0.38}$
HOPS-56_3RMS,2.343e-05,$0.55^{+0.13}_{-0.10}$,$0.39^{+0.17}_{-0.10}$,$0.59^{+0.12}_{-0.10}$,$0.37^{+0.07}_{-0.05}$,$0.50^{+0.13}_{-0.14}$,$0.66^{+0.17}_{-0.17}$,$0.57^{+0.11}_{-0.09}$,$0.52^{+0.12}_{-0.09}$
HOPS-140,1.537e-04,$nan^{+nan}_{-nan}$,$1.92^{+0.48}_{-0.40}$,$1.99^{+0.43}_{-0.37}$,$1.95^{+0.49}_{-0.41}$,$1.94^{+0.47}_{-0.40}$,$1.70^{+0.00}_{-0.07}$,$1.93^{+0.45}_{-0.37}$,$1.93^{+0.45}_{-0.37}$
HOPS-140_3RMS,2.262e-05,$0.54^{+0.13}_{-0.10}$,$0.36^{+0.17}_{-0.08}$,$0.58^{+0.12}_{-0.09}$,$0.36^{+0.07}_{-0.05}$,$0.48^{+0.13}_{-0.16}$,$0.63^{+0.18}_{-0.24}$,$0.55^{+0.11}_{-0.09}$,$0.51^{+0.12}_{-0.09}$


In [46]:
conditions = ['prop056_05myr','prop056_1myr','propn1_05myr', 'propn1_1myr']

for condition in conditions:
    table = mass_table_with_upper_lower_limit(condition=condition, mass_table_direc='data_result/mass_table', with_flux=True)
    write_latex_table(table, condition, data_origin='data')
    
    